In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install wikipedia-api
!pip install --upgrade openai
!pip install datasets
! pip install -U accelerate
! pip install -U transformers

In [ ]:
!pip install faiss-cpu

#Long From Content Generation Testing

In [5]:
# imports and inital setup
api_key = " "   # ADD YOUR OPENAI API KEY HERE
import wikipediaapi
from openai import OpenAI
import os
client = OpenAI(api_key)
from IPython.display import display, HTML
import re
import json
import torch
from sklearn.model_selection import train_test_split
import datasets
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification
from transformers import BartTokenizer
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments
from transformers import AdamW

import faiss
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import json

import nltk
nltk.download('punkt')
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Wiki API setup for Wikipedia Article Scraping

In [ ]:
wiki_wiki = wikipediaapi.Wikipedia('LongFromTextGen (ADD YOUR EMAIL HERE)', 'en')

# # # loading style of wiki pages to test zero shot generation
wikistyle = wiki_wiki.page('Wikipedia:Manual of Style')
print("Page - Exists: %s" % wikistyle.exists())
# print("Page - Summary: %s" % wikistyle.summary)

Page - Exists: True


In [ ]:
# loading style of verbiage
wikiverbiage = wiki_wiki.page('Wikipedia:Manual of Style/Words to watch')
print("Page - Exists: %s" % wikiverbiage.exists())
# print("Page - Summary: %s" % wikiverbiage.summary)

Page - Exists: True


In [ ]:
# loading writing guide
writingguide = wiki_wiki.page('Wikipedia:Writing better articles')
print("Page - Exists: %s" % writingguide.exists())
# print("Page - Summary: %s" % writingguide.summary)

Page - Exists: True


In [ ]:
neutralpov = wiki_wiki.page('Wikipedia:Neutral_point_of_view')
print("Page - Exists: %s" % neutralpov.exists())
# print("Page - Summary: %s" % neutralpov.summary)

Page - Exists: True


# Zero Shot Prompting Testing: (Can minimize only for refernce)

## Load Data To Compare Generated Output

In [ ]:
page_jimi = wiki_wiki.page('Jimi Hendrix')
print("Page - Exists: %s" % page_jimi.exists())
print("Page - Summary: %s" % page_jimi.summary)
print(page_jimi.text)

In [ ]:
system_prompt = f"""You are a large language model tasked with generating stylistically faithful
    and factually accurate sections of Wikipedia articles. The Wikipedia Manual of Style is as follows:

    {wikistyle.summary}
    {wikiverbiage.summary}
    {writingguide.summary}
    {neutralpov.summary}

    Please strictly adhere to the Manual of Style in generating Wikipedia sections."""

def create_passage_prompt(page_topic, section_topic):
    prompt = f"""Please generate textual content (paragraphs) for a section within a Wikipedia article.
    The title of the Wikipedia article is {page_topic}, and the section is {section_topic}.
    """
    return prompt


user_test_prompt = create_passage_prompt('Jimi Hendrix', 'Legacy')

In [ ]:
###### Charge per run so be CAREFUL when running this block!!!!!!!!

# chat_completion = client.chat.completions.create(
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": user_test_prompt}
#     ],
#     model="gpt-4-0125-preview",
#     max_tokens=1000,
#     n=1
# )

In [ ]:
print(chat_completion.choices[0].message.content)

## Test 1: Jimi Hendrix Description

Prompt used: user_test_prompt = create_passage_prompt('Jimi Hendrix', 'Description')

### Description

Jimi Hendrix, born Johnny Allen Hendrix on November 27, 1942, in Seattle, Washington, is widely regarded as one of the most influential and innovative guitarists in the history of popular music. Hendrix's style was characterized by his expert use of overdriven amplifiers with high volume and gain, a pioneering approach to feedback and distortion, and his creative use of effects units like the wah-wah pedal and the Uni-Vibe, effectively crafting a rich and expansive guitar soundscape that was unheard of at the time. His playing encompassed a variety of styles, ranging from blues, rhythm and blues, and soul to hard rock, jazz, and psychedelia, showcasing his versatility and profound impact on different music genres.

Hendrix's onstage presence was as captivating as his music. Flamboyantly dressed, often donning colorful and intricately designed costumes, he commanded the stage with an intense, energetic performance style. His dynamic live shows were marked by his physical engagement with his guitar, utilizing teeth and behind-the-back playing techniques, which, combined with his groundbreaking use of feedback and distortion, created a visually and sonically explosive experience for his audiences. Hendrix's performances at the Monterey Pop Festival in 1967, where he famously set his guitar on fire, and at Woodstock in 1969, culminating in an iconic rendition of "The Star-Spangled Banner", are emblematic of his profound influence on the performance art of rock music.

Beyond his extraordinary technical skills and performance artistry, Hendrix was also an innovative songwriter and producer. His studio work was characterized by its experimental approach to recording techniques, using the studio as an instrument to achieve his vision of a soundscape that melded his wide range of musical influences. Albums like "Are You Experienced", "Axis: Bold as Love", and "Electric Ladyland" were groundbreaking in their use of stereo effects, layered guitar tracks, and ambient sounds, influencing the development of several music genres and the work of countless musicians and producers.

Hendrix's impact on music extended far beyond his early death at the age of 27. His approach to the electric guitar, as both an instrument and an expressive tool, revolutionized popular music. He expanded the vocabulary of the electric guitar more than any other musician of his era or since, merging technical prowess with emotive expression to create a body of work that remains pivotal and influential in the history of music. Hendrix's legacy is that of a visionary artist whose innovations paved the way for future generations of musicians, reshaping the soundscape of rock and roll and the wider musical landscape in the process.


## Test 2: Career

Prompt used: user_test_prompt = create_passage_prompt('Jimi Hendrix', 'Career')

### Career

#### Early Years

Jimi Hendrix's music career began in the early 1960s, playing backup for various musicians of the time including the Isley Brothers and Little Richard, among others. His tenure with these acts honed his skills and stage presence, setting the stage for his later success as a solo artist. Despite his burgeoning talent, Hendrix found it difficult to achieve substantial success in the United States and in 1966, he moved to London. It was there, in the vibrant London music scene, that Hendrix's career would truly begin to flourish.

#### The Jimi Hendrix Experience

In late 1966, after being discovered by manager Chas Chandler of the Animals, Jimi Hendrix formed the Jimi Hendrix Experience with bassist Noel Redding and drummer Mitch Mitchell. The trio quickly made a name for themselves in the UK, with their first single, "Hey Joe," reaching the top ten in the British charts. The release of their debut album, "Are You Experienced," in 1967, revolutionized rock music with its innovative guitar work and experimental soundscapes, showcasing Hendrix's extraordinary talent and creativity. The album included timeless tracks such as "Purple Haze," "The Wind Cries Mary," and the title track "Are You Experienced."

#### Monterey Pop Festival and Woodstock

The band's international breakthrough came with their electrifying performance at the Monterey Pop Festival in 1967, during which Hendrix famously set his guitar on fire. This moment not only solidified his status as a guitar icon but also marked the beginning of his influence on the rock genre as a whole. In 1969, Hendrix's legendary performance of the Star-Spangled Banner at the Woodstock Festival became emblematic of the 1960s counterculture, showcasing his skilled manipulation of guitar feedback and distortion to make profound political and social commentary.

#### Band of Gypsys and Later Work

Following the dissolution of the Jimi Hendrix Experience in 1969, Hendrix formed a new band, Band of Gypsys, with bassist Billy Cox and drummer Buddy Miles. The band explored more funk and soul influences, as demonstrated in their live album "Band of Gypsys," which featured the track "Machine Gun," a powerful anti-war anthem. Despite the brief tenure of Band of Gypsys, this period is noted for displaying Hendrix's growing interest in different musical genres and a deeper lyrical introspection.

#### Legacy and Influence

Jimi Hendrix's career, though notably short following his untimely death in 1970, left an indelible mark on the world of music. His innovative use of electric guitar techniques, such as feedback, wah-wah, and fuzz distortion, alongside his pioneering use of the recording studio as an instrument, has influenced countless musicians and genres. Beyond his technical proficiency, Hendrix's emotional depth and compositional skill have solidified his status as one of the greatest musicians in the history of rock music. His work continues to inspire new generations of artists across various music styles, reaffirming his timeless appeal and significance in the music world.


## Test 3: Legacy

prompt used: user_test_prompt = create_passage_prompt('Jimi Hendrix', 'Legacy')

### Legacy

Jimi Hendrix, widely regarded as one of the most influential electric guitarists in the history of popular music, and one of the most celebrated musicians of the 20th century, left an indelible mark on the world of rock music and beyond. His innovative techniques and pioneering use of tone-altering effects like wah-wah and distortion fundamentally altered the sound of rock and roll. Hendrix's legacy is not only preserved through his recordings but also through his influence on generations of guitarists and musicians who have drawn inspiration from his work.

Despite his meteoric career being cut short by his untimely death at age 27, Hendrix was posthumously awarded numerous accolades. These include induction into the Rock and Roll Hall of Fame in 1992 and the UK Music Hall of Fame in 2005. Rolling Stone magazine ranked him as the greatest guitarist and the sixth greatest artist of all time. His music continues to sell millions of copies yearly, attesting to the enduring appeal of his artistry.

Hendrix's innovative approach to guitar playing has inspired a diverse range of musicians across various genres. From his explosive performances at the Monterey Pop Festival and Woodstock to his groundbreaking studio albums like "Are You Experienced", "Axis: Bold as Love", and "Electric Ladyland", Hendrix's work pushed the boundaries of what was thought possible in music. His combination of raw emotion, technical prowess, and experimental soundscapes has made his work a cornerstone of modern music.

Moreover, Hendrix's influence extends beyond his guitar work. His expressive use of feedback, pioneering recording techniques, and his embracing of rock as a form of personal and artistic expression have influenced countless artists. His cultural impact can be seen in how he challenged norms and conventions of his time, from his fashion sense to his attitudes towards race and identity. Hendrix’s life and music continue to inspire discussions on cultural, social, and musical innovation.

Today, Hendrix's legacy is kept alive through various tributes, including music festivals, academic studies, and exhibitions at the Experience Music Project Museum in Seattle. His childhood home has become a place of pilgrimage for fans, and his music is a staple of guitar curricula worldwide. Jimi Hendrix's influence on music and culture is immortal, reverberating through the ages and continuing to inspire new generations of fans and musicians alike.

Jimi Hendrix transformed the electric guitar's role in music, from a rhythmic background instrument to a powerful vehicle of personal expression. Through his mastery and innovation, Hendrix invited the world to listen to the guitar in an entirely new way, ensuring his legacy would resonate throughout the annals of music history.


## Adapting IRP into wiki dataset:

In [ ]:
def clean_text(text):
    # Remove all punctuation except ., ?, and ! that appear to be at the end of sentences
    # This pattern will remove punctuation except for ., ?, and ! followed by a space, the end of the text, or a quotation mark
    clean_text = re.sub(r'(?<!\d)([^\w\s\.?!])|(?<=[\w])([\.?!])(?=[^\s"\'])', '', text)
    return clean_text

# Test the function
text = "Hello, world! This is a test... What will happen? Let's see: will it work? Yes! It works."
cleaned_text = clean_text(text)
print(cleaned_text)

Hello world! This is a test.. What will happen? Lets see will it work? Yes! It works.


In [ ]:
parsed_guitarist_articles = []

# List of guitarist Wikipedia page titles
guitarists = ['Jimi Hendrix', 'Eddie Van Halen', 'Eric Clapton', 'B. B. King', 'Jimmy Page', 'Francisco Tárrega', 'Chuck Berry', 'Stevie Ray Vaughan',
              'Slash (musician)', 'Carlos Santana', 'Keith Richards', 'George Harrison', 'Pete Townshend', 'Ana Vidović', 'Andrés Segovia',
              'Billy Idol', 'Leo Brouwer', 'John Mayer', 'Chet Atkins', 'Billie Joe Armstrong', 'Matt Bellamy', 'Ed Sheeran', 'David Gilmour',
              'Gary Green', 'Kurt Cobain', 'Mark Knopfler', 'Elvis Presley', 'Julian Bream', 'Tommy Emmanuel', 'Don Felder', 'Sunil Ganguly',
              'Kirk Hammett', 'Matt Heafy', 'Tony Iommi', 'Robert Johnson', 'Sungha Jung', 'Ed King', 'Chino Moreno', 'Duff McKagan', 'Dave Navarro',
              'Tom Petty', 'Iggy Pop', 'Katy Perry', 'Olivia Rodrigo', 'Johnny Ramone', 'Kapil Srivastava', 'Taylor Swift', 'Hillel Slovak', 'Paul Tobias ',
              'Keith Urban', 'Björn Ulvaeus', 'Vinnie Vincent', 'Jacky Vincent', 'Paul Waaktaar-Savoy', 'Andrew York (guitarist)', 'Chris Martin',
              'Thom Yorke', 'Angus Young', 'Lee Ving', 'John Lennon', 'Fernando Sor', 'Tom Morello', 'Adam Levine', 'Bernie Leadon', 'Henry McCullough',
              'Eric Johnson', 'Tracii Guns', 'Synyster Gates', 'Zach Filkins', 'William DuVall', 'Allen Collins', 'Paul Banks', 'David Byrne', 'Jeff Beck',
              'Mike Campbell', 'David Crosby', 'Bob Dylan', 'Roland Dyens', 'Steve Earle', 'Robb Flynn', 'John Frusciante', 'Rita Hosking', 'Ted Nugent',
              'Adrian Smith', 'Pat Smear', 'Devin Townsend', 'Pete Townshend', 'John Petrucci', 'Steve Jones', 'Steve Hunter', 'Ed O\'Brien', 'Aaron North',
              'Steve Morse', 'Paul McCartney', 'Phil Manzanera', 'Madonna', 'Mark Knopfler', 'Mick Jones (The Clash guitarist)', 'Susanna Hoffs', 'Dan Auerbach']
print(len(guitarists))
# for guitarist in guitarists:
#     page = wiki_wiki.page(guitarist)
#     if page.exists():
#         print('success: adding summary of ' + guitarist)
#         # Clean the page's text and store it
#         cleaned_text = clean_text(page.summary)
#         parsed_guitarist_articles.append(cleaned_text)

100


In [ ]:
parsed_guitarist_articles[2]

In [ ]:
json_data = json.dumps(parsed_guitarist_articles)
# To save the JSON data to a file
with open('guitarist_articles.json', 'w') as file:
    file.write(json_data)


with open('guitarist_articles.json', 'r') as file:
        parsed_guitarsit_json = json.load(file)

In [ ]:
guitarist_train, guitarist_tesst = train_test_split(parsed_guitarsit_json, test_size=0.3, random_state=42)

In [ ]:
guitarist_tesst

## Fine Tuning GPT2 (initally testing with GPT2 before moving to newer models)

In [ ]:
data = {"text": guitarist_train}
dataset = datasets.Dataset.from_dict(data)
model_name = 'gpt2-medium'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize_and_add_labels(examples):
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(examples["text"], padding='max_length', truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_add_labels, batched=True)

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

In [ ]:
# !pip install accelerate -U
# !pip install transformers[torch]
# ! pip install -U accelerate
# ! pip install -U transformers

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    num_train_epochs=10,  # number of training epochs
    per_device_train_batch_size=1,  # batch size
    per_device_eval_batch_size=1,
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()
trainer.save_model('/content/drive/MyDrive/FDLResearch/guitarist_imitator0')

Step,Training Loss


##Test Inference

In [ ]:
model_path = '/content/drive/MyDrive/FDLResearch/guitarist_imitator0'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

In [ ]:
guitarist_tesst

In [ ]:
prompt = "Jimi Hendrix was "

In [ ]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [ ]:
# Set the maximum length of the sequence and the number of generated sequences
max_length = 500
num_return_sequences = 1

# Generate text
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=max_length + len(input_ids[0]),
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample=True,
    num_return_sequences=num_return_sequences,
)

# Decode the generated sequences to text
generated_texts = [tokenizer.decode(generated_sequence, skip_special_tokens=True) for generated_sequence in output_sequences]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


####Inference 1: Prompt: "Ed Sheeran was born on "

In [ ]:
generated_texts

['Ed Sheeran was born on August 27, 1989 in London, England and raised at the Royal Academy of Music. He is an accomplished songwriter with a number to his name including "Blank Space", which won Best Pop Vocal Album for its single "...Ready For It?", as well several Grammy Awards (Best Male Solo Performance by Ansel Elgort, Songwriter of 2015 ) and two Brit Award nominations. His debut album The National received critical acclaim but did not achieve commercial success due mainly because of poor sales figures; however it sold over one million copies worldwide during release week']

####Inference 2: Prompt: "Billie Joe Armstrong is "

In [ ]:
generated_texts

["Billie Joe Armstrong is \xa0a singer and songwriter from Chicago who has released several albums under the name of his own band The Animals. His debut album, 2001's \xa0 Animal, was a critical success with over one million copies sold in its first week on the market. In 2003 he began work on another studio effort called 2005's Wildflower which featured guest appearances by Stevonnie Wonder (as well as other artists such Asleep at My Feet ) along side members of Led Zeppelin III"]

####Infernce 3: Prompt: "Edward Van Halen was "
- Modified max output to 1000 words

In [ ]:
generated_texts

["Edward Van Halen was \xa0a singer and songwriter who played guitar, mandolin, bass guitar & drums in the rock band Led Zeppelin. He also wrote songs for movies such as The Rocketeer (1962) and A Hard Day's Night to Remember \xa0 which were all nominated for an Academy Award."]

####Inference 4: Prompt: "Jimi Hendrix was "

In [ ]:
generated_texts

['Jimi Hendrix was \xa0a guitarist and songwriter who became a pop star in the 1960s with his hit singles " I Want to Hold Your Hand " (1965), "(You Make Me Feel Like You Do)" \xa0and " Rock \'n Roll Suicide Song". He also wrote songs for such artists as Jimmie Vaughan, John Mayall Jr., Bob Marley & The Wailers,, Elvis Presleys and many others. His music has been described by Rolling Stone magazine :\nHe is one of the most influential guitarists ever born; he\'s an iconoclast whose influence on rock history extends far beyond popular culture']

## Modifying DataSet to Include Full Text and not just summary

In [ ]:
parsed_guitarist_articles2 = []

# List of guitarist Wikipedia page titles
guitarists = ['Jimi Hendrix', 'Eddie Van Halen', 'Eric Clapton', 'B. B. King', 'Jimmy Page', 'Francisco Tárrega', 'Chuck Berry', 'Stevie Ray Vaughan',
              'Slash (musician)', 'Carlos Santana', 'Keith Richards', 'George Harrison', 'Pete Townshend', 'Ana Vidović', 'Andrés Segovia',
              'Billy Idol', 'Leo Brouwer', 'John Mayer', 'Chet Atkins', 'Billie Joe Armstrong', 'Matt Bellamy', 'Ed Sheeran', 'David Gilmour',
              'Gary Green', 'Kurt Cobain', 'Mark Knopfler', 'Elvis Presley', 'Julian Bream', 'Tommy Emmanuel', 'Don Felder', 'Sunil Ganguly',
              'Kirk Hammett', 'Matt Heafy', 'Tony Iommi', 'Robert Johnson', 'Sungha Jung', 'Ed King', 'Chino Moreno', 'Duff McKagan', 'Dave Navarro',
              'Tom Petty', 'Iggy Pop', 'Katy Perry', 'Olivia Rodrigo', 'Johnny Ramone', 'Kapil Srivastava', 'Taylor Swift', 'Hillel Slovak', 'Paul Tobias ',
              'Keith Urban', 'Björn Ulvaeus', 'Vinnie Vincent', 'Jacky Vincent', 'Paul Waaktaar-Savoy', 'Andrew York (guitarist)', 'Chris Martin',
              'Thom Yorke', 'Angus Young', 'Lee Ving', 'John Lennon', 'Fernando Sor', 'Tom Morello', 'Adam Levine', 'Bernie Leadon', 'Henry McCullough',
              'Eric Johnson', 'Tracii Guns', 'Synyster Gates', 'Zach Filkins', 'William DuVall', 'Allen Collins', 'Paul Banks', 'David Byrne', 'Jeff Beck',
              'Mike Campbell', 'David Crosby', 'Bob Dylan', 'Roland Dyens', 'Steve Earle', 'Robb Flynn', 'John Frusciante', 'Rita Hosking', 'Ted Nugent',
              'Adrian Smith', 'Pat Smear', 'Devin Townsend', 'Pete Townshend', 'John Petrucci', 'Steve Jones', 'Steve Hunter', 'Ed O\'Brien', 'Aaron North',
              'Steve Morse', 'Paul McCartney', 'Phil Manzanera', 'Madonna', 'Mark Knopfler', 'Mick Jones (The Clash guitarist)', 'Susanna Hoffs', 'Dan Auerbach']
print(len(guitarists))
for guitarist in guitarists:
    page = wiki_wiki.page(guitarist)
    if page.exists():
        print('success: adding summary of ' + guitarist)
        # Clean the page's text and store it
        cleaned_text = clean_text(page.text)
        parsed_guitarist_articles2.append(cleaned_text)

In [ ]:
parsed_guitarist_articles2[0]

'James Marshall Jimi Hendrix born Johnny Allen Hendrix November 27, 1942  September 18, 1970) was an American guitarist songwriter and singer. Although his mainstream career spanned only four years he is widely regarded as the greatest and one of the most influential electric guitarists in the history of popular music and one of the most celebrated musicians of the 20th century. The Rock and Roll Hall of Fame describes him as arguably the greatest instrumentalist in the history of rock music.Born in Seattle Washington Hendrix began playing guitar at age 15. In 1961, he enlisted in the US Army but was discharged the following year. Soon afterward he moved to Clarksville then Nashville Tennessee and began playing gigs on the chitlin circuit earning a place in the Isley Brothers backing band and later with Little Richard with whom he continued to work through mid1965. He then played with Curtis Knight and the Squires before moving to England in late 1966 after bassist Chas Chandler of the

In [ ]:
# json_data2 = json.dumps(parsed_guitarist_articles2)
# # To save the JSON data to a file
# with open('/content/drive/MyDrive/FDLResearch/guitarist_articlesFull.json', 'w') as file:
#     file.write(json_data2)


with open('/content/drive/MyDrive/FDLResearchSchool/LongFromExploration/guitarist_articlesFull.json', 'r') as file:
        parsed_guitarsit_json2 = json.load(file)

In [ ]:
guitarist_train_full, guitarist_test_full = train_test_split(parsed_guitarsit_json2, test_size=0.4, random_state=42)

In [ ]:
data = {"text": guitarist_train_full}
dataset = datasets.Dataset.from_dict(data)
model_name = 'gpt2-medium'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
tokenized_dataset_full = dataset.map(tokenize_and_add_labels, batched=True)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

In [ ]:
training_args2 = TrainingArguments(
    output_dir="/content/drive/MyDrive/FDLResearch/gpt2-finetuned",
    num_train_epochs=10,  # number of training epochs
    per_device_train_batch_size=1,  # batch size
    per_device_eval_batch_size=1,
    warmup_steps=200,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='/content/drive/MyDrive/FDLResearch/logs',
)

trainer = Trainer(
    model=model,
    args=training_args2,
    train_dataset=tokenized_dataset_full,
)

In [ ]:
trainer.train()
trainer.save_model('/content/drive/MyDrive/FDLResearch/guitarist_imitatorFull')

Step,Training Loss
500,2.694400


In [ ]:
model_path = '/content/drive/MyDrive/FDLResearch/guitarist_imitatorFull'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

In [ ]:
guitarist_test_full

In [ ]:
prompt = "Roland Dyens: Early Life, Career, Influences, Legacy"

In [ ]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [ ]:
# Set the maximum length of the sequence and the number of generated sequences
max_length = 1000
num_return_sequences = 1

# Generate text
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=max_length + len(input_ids[0]),
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample=True,
    num_return_sequences=num_return_sequences,
)

# Decode the generated sequences to text
generated_texts = [tokenizer.decode(generated_sequence, skip_special_tokens=True) for generated_sequence in output_sequences]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


##Inference 1: Prompt: "Leo Brouwer was "

In [ ]:
generated_texts

["Leo Brouwer was born on May 20, 1947 in New York City and grew up near the city's financial district. He is an actor known for his work with The Turtles (1992), My Fair Lady(1995)and InglouriousBastards2: Battle of Olympus () as well a recurring role opposite John Candy in Dallas Buyers Club. His other credits include How to Train Your Dragon 2 (2010); American Pie (2013; directed by Steven Spielberg ) ; Pirates of Penzance (?, 2015 ); Star Trek Into Darkness : Beyond Earth (2015), Indiana Jones And The Kingdom Of Thieves (2016 ), Jurassic World (2017)..."]

###Inference 2: Prompt = "Leo Brouwer: Life, Career, Legacy"

In [ ]:
generated_texts

['Leo Brouwer: Life, Career, Legacy\n\nBorn in New York City on May 21st 1945 Leo was the youngest of three children born to a successful newspaper reporter and his wife. He attended Stuyvesant High School where he developed an interest for music which led him into jazz at age 16 when fellow students introduced himself as "The Man Who Licked His Shoes". After graduating from Columbia University with a degree In Music Production (MSP) it was during this time that John Coltrane began playing guitar along side Miles Davis who would later become one half brother of George Harrison. While still attending college under Brian Jones, guitarist Jack Bruce joined forces again after hearing about another young musician named Charlie Parker whom they both admired but were not close enough friends or fans yet so decided to join up instead leaving them behind while traveling around Europe together until their return home late last year following two years abroad before returning back across North Am

####Infernece 3: Prompt: "Eric Clapton: Early Life, Career, Influences, Legacy"
- also changed max otuput length to 10000 for this run

In [ ]:
generated_texts

['Eric Clapton: Early Life, Career, Influences, Legacy\nClint Eastwood and his wife Linda were born on November 21st 1965 in London England to the late John (Linda) Westley and Margaret Ann Gibson. They had two older brothers who also played guitar but they did not have a father figure as their mother was divorced from her husband of twenty years when he died leaving them with only one brother at home during this time.[1] Their parents separated after four years so Clint moved into an apartment where she worked until age eleven while playing piano for local bands such The Beatles[2][3]. He later attended St Paul\'s School before attending King Edward VII High School which is located near Manchester United Football Club that year graduating first class along side fellow student Jack Nicholson whom Bruce Springsteen has said inspired him greatly due both being influenced by rock music early life styles like bluesrock or countrypop In high school Cliff began performing live shows around t

#### Inference 4: Prompt: "Roland Dyens: Early Life, Career, Influences, Legacy"

In [ ]:
generated_texts

# IRP Implementation: (Can minimize only for refernce)

## Retriever Adaptation:

In [ ]:
import json

In [ ]:
with open('/content/drive/MyDrive/LongFromExploration/guitarist_articlesFull.json') as f:
    data = json.load(f)

# Initialize empty lists to hold all sentences across documents and their positions
# all_sentences = []
# document_positions = []

# # Iterate over each document (string) in the list
# for document in data:
#     sentences = sent_tokenize(document)
#     all_sentences.extend(sentences)
#     document_positions.append(len(all_sentences))

# print(all_sentences[:2])

In [ ]:
# Assuming 'data' is your list of documents
document_positions = [0]  # Start with 0 for the first document
for document in data:
    sentences_in_doc = sent_tokenize(document)
    # For each document, add the count of sentences to the last element to get positions
    document_positions.append(document_positions[-1] + len(sentences_in_doc))

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

inputs = tokenizer(all_sentences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
position_labels = []
for idx, _ in enumerate(all_sentences):
    position_labels.append(np.digitize(idx, document_positions, right=True))

In [ ]:
class SentencePositionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: self.encodings[key][idx].clone().detach() for key in self.encodings}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
position_labels = torch.tensor(position_labels, dtype=torch.long)
dataset = SentencePositionDataset(inputs, position_labels)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

<ipython-input-103-6ad9c72093a2>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  position_labels = torch.tensor(position_labels, dtype=torch.long)


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=max(document_positions)+1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
print(len(inputs['input_ids']), len(position_labels))

18519 18519


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    # Wrap loader with tqdm for a progress bar
    progress_bar = tqdm(loader, desc=f"Epoch {epoch+1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Update the progress bar with the current loss
        progress_bar.set_postfix({'avg_loss': total_loss / (progress_bar.n + 1)})
    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss}")


In [ ]:
model_dir = "/content/drive/MyDrive/LongFromExploration/retrieverFull"
# Create the directory if it doesn't exist
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save the model
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

## Paraphraser Adaptation:


In [ ]:
import json
from nltk.tokenize import sent_tokenize
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def get_sentence_embedding(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states
    last_layer = hidden_states[-1]
    sentence_embedding = torch.mean(last_layer, dim=1).squeeze()
    return sentence_embedding.cpu().numpy()

In [ ]:
def find_most_similar(embedding, embedding_dict):
    # Convert embedding_dict values to a matrix (each row is an embedding)
    other_embeddings = np.array(list(embedding_dict.values())).astype('float32')

    # Ensure the target embedding is 2D
    if embedding.ndim == 1:
        embedding = embedding.reshape(1, -1)

    # Calculate cosine similarity between the target embedding and all other embeddings
    similarities = cosine_similarity(embedding, other_embeddings).flatten()

    # Find the index of the sentence with the highest similarity
    most_similar_index = similarities.argmax()

    # Retrieve the most similar sentence using the index
    most_similar_sentence = list(embedding_dict.keys())[most_similar_index]
    return most_similar_sentence

In [ ]:
# Assuming parsed_guitarist_articles2 is a list of topics corresponding to each document in 'data'
document_topic_pairs = [(doc, parsed_guitarist_articles2[idx]) for idx, doc in enumerate(data)]


In [ ]:
# Compute sentence embeddings for each sentence in each document
sentence_embeddings = {}
for doc, _ in tqdm(document_topic_pairs):
    sentences = sent_tokenize(doc)
    for sentence in sentences:
        sentence_embeddings[sentence] = get_sentence_embedding(sentence, model, tokenizer)


100%|██████████| 100/100 [01:56<00:00,  1.16s/it]


In [ ]:
all_sentences = [sentence for document in data for sentence in sent_tokenize(document)]

In [ ]:
fact_embeddings = {}
for sentence in tqdm(all_sentences):
    fact_embeddings[sentence] = get_sentence_embedding(sentence, model, tokenizer)

100%|██████████| 18519/18519 [01:58<00:00, 156.12it/s]


In [ ]:
embeddings_matrix = np.array(list(fact_embeddings.values())).astype('float32')

# Create a FAISS index
dim = embeddings_matrix.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings_matrix)  # Add the sentence embeddings to the index

In [ ]:
k = 5  # Plus one, since the closest sentence might be `z` itself

quadruples = []

for doc, topic in tqdm(document_topic_pairs):
    sentences = sent_tokenize(doc)
    for sentence in sentences:
        embedding = sentence_embeddings[sentence].astype('float32').reshape(1, -1)

        # Search for the top k + 1 similar sentences because one of them will be `z` itself
        D, I = index.search(embedding, k + 1)

        # Extracting indices while excluding the first one (itself) and finding 'y'
        y_indices = [i for i in I.flatten() if list(fact_embeddings.keys())[i] != sentence][:k]
        facts_for_sentence = [list(fact_embeddings.keys())[i] for i in y_indices]

        # 'y' can be the most similar sentence other than 'z' itself or a placeholder if not found
        y = facts_for_sentence[0] if facts_for_sentence else "Placeholder for similar sentence"

        quadruples.append((topic, facts_for_sentence, y, sentence))


100%|██████████| 100/100 [00:01<00:00, 55.67it/s]


In [ ]:
formatted_data = torch.load('/content/drive/MyDrive/LongFromExploration/formatted_data.pt')

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device)

# formatted_data = []
# for topic, facts, y, z in tqdm(quadruples):
#     input_text = f"<topic> {topic} </topic> {' '.join([f'<fact> {fact} </fact>' for fact in facts])} <plan> {y} </plan>"
#     target_text = z

#     # Tokenization and ensuring tensors are on the correct device
#     input_ids = tokenizer.encode(input_text, truncation=True, padding="max_length", max_length=512)
#     target_ids = tokenizer.encode(target_text, truncation=True, padding="max_length", max_length=128)

#     formatted_data.append({"input_ids": input_ids, "labels": target_ids})

# Custom Dataset
class BartDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# DataLoader
dataset = BartDataset(formatted_data)
loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [ ]:
torch.save(formatted_data, '/content/drive/MyDrive/LongFromExploration/formatted_data.pt')

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model.to(device)

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/FDLResearch/LongFromExploration/results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/FDLResearch/LongFromExploration/logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
500,3.184000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=870, training_loss=2.1692468402029452, metrics={'train_runtime': 849.3164, 'train_samples_per_second': 65.414, 'train_steps_per_second': 1.024, 'total_flos': 1.693756420521984e+16, 'train_loss': 2.1692468402029452, 'epoch': 3.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/LongFromExploration/paraphraserfull_0")

## Putting it Together:

In [ ]:
imitator_path = '/content/drive/MyDrive/LongFromExploration/guitarist_imitatorFull'
imitator = GPT2LMHeadModel.from_pretrained(imitator_path)
imitator_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

In [ ]:
retriever_dir = '/content/drive/MyDrive/LongFromExploration/retrieverFull'
retriever = DistilBertForSequenceClassification.from_pretrained(retriever_dir)

# Load the tokenizer
retriever_tokenizer = DistilBertTokenizer.from_pretrained(retriever_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
retriever.to(device)

In [ ]:
paraphraser_dir = '/content/drive/MyDrive/LongFromExploration/paraphraserfull_0'
paraphraser = BartForConditionalGeneration.from_pretrained(paraphraser_dir)
paraphraser_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
prompt = "Jimi Hendrix was a "
topic = "Jimi Hendrix"

In [ ]:
input_ids = imitator_tokenizer.encode(prompt, return_tensors='pt')

In [ ]:

max_length = 500
num_return_sequences = 1


content_plans = imitator.generate(
    input_ids=input_ids,
    max_length=max_length, # + len(input_ids[0]) commented to limit topic wandering
    temperature=0.6,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
    do_sample=True,
    num_return_sequences=num_return_sequences,
)


content_plan = imitator_tokenizer.decode(content_plans[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
content_plan

'Jimi Hendrix was a American guitarist and singer. He first played with the group Green Day from 1975 to 1981, then in 1983 he joined Rage Against The Machine as part of their eponymous lineupAfter leaving Rage A&M his solo career spanned more than 20 yearsFrom founding member Dave Grohl California punk band Screaming Trees 1970s grunge outfit Deftones 1980) thru vocalist Jason Derulo Detroit hardcore groups Dillinger Four 1984–1997; now lead guitar player for hiphop duo Flying Colors 2016).\nIn 1994, while still an active college student at Ohio University John Hammond formed Gorillaz which later became one half oracles Jonny Greenwood bass drumming vocals Electric Light Orchestra keyboards synthesizers backing singers Duff McKagan drums saxophone percussion David Gilmours mandolin<|endoftext|>'

In [ ]:
fact_sentences = all_sentences

In [ ]:
k=15
result = ""
for y in sent_tokenize(content_plan):
  y_embedding = get_sentence_embedding(y, retriever, retriever_tokenizer)
  if y_embedding.ndim == 1:
    y_embedding = y_embedding.reshape(1, -1)
  _, I = index.search(y_embedding, k)
  x = [fact_sentences[i] for i in I[0]]
  t = topic

  formatted_t = f"<topic> {t} </topic>"
  formatted_x = " ".join([f"<fact> {fact} </fact>" for fact in x])
  formatted_y = f"<plan> {y} </plan>"
  input_text = f"{formatted_t} {formatted_x} {formatted_y}"

  print(input_text)

  input_tokenized = paraphraser_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
  inputs = {k: v.to(model.device) for k, v in input_tokenized.items()}
  paraphraser.to(inputs['input_ids'].device)

  output = paraphraser.generate(**inputs, max_length=128, num_beams=5, early_stopping=False)
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  result += (" " + generated_text)

print(result)


### Inference 1: prompt = "Jimi Hendrix was", Topic = "Jimi Hendrix"

<topic> Jimi Hendrix </topic> <fact> It leaned heavily towards American roots music  folk blues and country music. </fact> <fact> The studio officially opened for business on August 25, and a grand opening party was held the following day. </fact> <fact> In 2020, Santana launched his own brand of cannabis named Mirayo that honours the spiritual and ancient Latin heritage of the plant.From 1973 to 2007, he was married to Deborah King daughter of blues musician Saunders King. </fact> <fact> Hendrixs arrival had an immediate and major effect on the next phase of Claptons career. </fact> <fact> But any bad feelings came from us being three guys who were traveling too hard getting too tired and taking too many drugs ... </fact> <fact> The first acts signed to the new label were Ravi Shankar and the duo Splinter. </fact> <fact> His new guitar for this period was the Epiphone Love Death Baritone Flying V along with a custom Explorer which he occasionally used on the Locust tour. </fact> <fact> After two years of litigation the parties agreed to a resolution that granted Chalpin the distribution rights to an album of original Hendrix material. </fact> <fact> Both drank heavily and had fiery tempers. </fact> <fact> Thus the Jimi Hendrix Experience performed their first show on October 13, 1966, at the Novelty in Evreux. </fact> <fact> On his 11th birthday Bream was given a small gutstrung Spanish guitar by his father. </fact> <fact> Soon afterward Hendrix joined Little Richards touring band the Upsetters. </fact> <fact> She plays Americana and bluegrass styles of music with both standards and original songs. </fact> <fact> His father Anthony John Martin of Whitestone House Exeter is a retired chartered accountant and his mother Alison Martin who is from Zimbabwe is a music teacher. </fact> <fact> Writing credits were more collaborative and the bands sound had matured into more than what was strictly considered fusion at the time. </fact> <plan> Jimi Hendrix was born on July 18, 1964 in Los Angeles California. </plan>
<topic> Jimi Hendrix

### Infernence 2: prompt = "Allen Collins was ", topic = "Allen Collin"

<topic> Allen Collin </topic> <fact> I used to go and pick them up. </fact> <fact> Brian Haners BlowUp Doll music video. </fact> <fact> Foxboro Hot Tubs released one album Stop Drop and Roll!! </fact> <fact> I learned that you have got to work at it. </fact> <fact> Before that Vaughan had briefly used other drugs such as cannabis methamphetamine and Quaaludes the brand name for methaqualone. </fact> <fact> In 2023, a species of fossil stingray from Italy was named Dasyomyliobatis thomyorkei in honor of Yorke. </fact> <fact> Instead of throwing the smashed parts away Townshend reassembled the pieces as a sculpture. </fact> <fact> ! </fact> <fact> Inspired by Reinhardts twofingered guitar playing Iommi decided to try playing guitar again though the injury made it quite painful to do so. </fact> <fact> Eventually former Stone Temple Pilots vocalist Scott Weiland joined the band. </fact> <fact> Although it was an option Iommi never seriously considered switching hands and learning to play righthanded. </fact> <fact> Morello responded to Meinert stating that he would show leniency and forgive Meinert if he embraced a 15 minimumwage. </fact> <fact> Leadon toured with the Eagles from 2013 through 2015 during their History of the Eagles Tour In 2015, Leadon appeared on producer Ethan Johns third solo album Silver Liner. </fact> <fact> He also had a short stint with the Nitty Gritty Dirt Band in the late 1980s. </fact> <fact> The following September the band released the longawaited albums Use Your Illusion I and Use Your Illusion II which debuted at No. </fact> <plan> Allen Collins was  born in Rome Italy and raised in the New York area. </plan>
<topic> Allen Collin </topic> <fact> Morello responded to Meinert stating that he would show leniency and forgive Meinert if he embraced a 15 minimumwage. </fact> <fact> In 2023, a species of fossil stingray from Italy was named Dasyomyliobatis thomyorkei in honor of Yorke. </fact> <fact> Although it was an option Iommi never seriously considered switching hands and learning to play righthanded. </fact> <fact> Leadon toured with the Eagles from 2013 through 2015 during their History of the Eagles Tour In 2015, Leadon appeared on producer Ethan Johns third solo album Silver Liner. </fact> <fact> I learned that you have got to work at it. </fact> <fact> Burn Halos Dirty Little Girl track and music video. </fact> <fact> I used to go and pick them up. </fact> <fact> Former Guns N Roses guitarist Izzy Stradlin was initially involved but left after the others decided to find a lead singer. </fact> <fact> ! </fact> <fact> This forced the tours cancellation. </fact> <fact> Eventually former Stone Temple Pilots vocalist Scott Weiland joined the band. </fact> <fact> The following September the band released the longawaited albums Use Your Illusion I and Use Your Illusion II which debuted at No. </fact> <fact> The second single So Far Under was written by DuVall who also played the guitar solo on the track. </fact> <fact> Foxboro Hot Tubs released one album Stop Drop and Roll!! </fact> <fact> Tony Mc Alpine Tour 2019. </fact> <plan> He is an American musician who performs guitar, bass clarinet saxophone banjo mandolin viola tenor piano synthesiser cello alto Spanish flamenco steel violin baritone harmonica bluegrass electric blues rock classical metal jazz fusion alternative) folk dance pop electronic music electronic sound design film scores art installations radio programs business competitions entertainment
Collins has released five solo albums three of which were reissues with bonus tracks including his debut album The Long Pond 2007). </plan>

### Inference 3: Prompt: Roland Dyens was, Topic: Roland Dyens

<topic> Roland Dyens </topic> <fact> He received degrees from James Madison University in Virginia and the University of Southern California USC. </fact> <fact> The Nightwatchman appeared on the albumDVD Axis Of Justice Concert Series Volume 1, contributing the songs Until the End The Road I Must Travel and Union Song. </fact> <fact> His vocal delivery ranges from screaming to an operaesque singing while his musical style is rooted in metal and his albums are written to express different aspects of his personality. </fact> <fact> His father is Irish and Sheeran has stated that his father is from a very large Catholic family. </fact> <fact> Boat was released as the second single off the record on 21 April 2023. </fact> <fact> Working with many projects simultaneously at that time Townsend stated in 2012 the Obviouser project is vying for pole position until he wakes up and says he wants to do it. </fact> <fact> He also wrote and played guitar on the tracks Dawn of a Golden Age I Dont Wanna Be A Superhero and Blood and Flames also contributing vocals to the latter. </fact> <fact> Townsend referred the music of the album to be closest to his heart at this point of his life and that it is an important and satisfying project he does not want to rush. </fact> <fact> Guns 2016-present
In the summer of 2016, Guns teamed up with Phil Lewis for the first time in 14 years for a handful of shows under the name LA. </fact> <fact> This song was included on Paisleys album Play and it went to No. </fact> <fact> Sheeran had initially planned to put the song on his third album  and added that the track would have been scrapped before Bieber took the song. </fact> <fact> He was joined by former judge Jennifer Lopez and newest judge Harry Connick Jr. On 23 June 2014, Fox announced that Urban would return to the judging panel for season 14. </fact> <fact> Other appearances
Heafy won the Metal Hammer Golden God award in 2006The same year he sang one song Blind for Korn at the Download Festival when Korns lead vocalist Jonathan Davis fell illHeafyalong with several other metal artistsmakes guest appearances in the music videos for Aesthetics of Hate by Machine Head All I Want by A Day to Remember and Moving On by Asking Alexandria. </fact> <fact> In September 2015, the company launched its first series of pedals surprising the industry. </fact> <fact> Eventually Guns settled on the idea of writing and recording an album of music that reminded him of what I used to listen to when I was a teenager first learning to play guitar. </fact> <plan> Roland Dyens was  born in Belfast on 27 August 1947) is an English musician songwriter and record producer best known as the founder and guitarist of progressive rock band Cream. </plan>
<topic> Roland Dyens </topic> <fact> In 2013, Armstrong and singersongwriter Norah Jones released the album Foreverly consisting of covers of songs from The Everly Brothers album Songs Our Daddy Taught Us. </fact> <fact> His new guitar for this period was the Epiphone Love Death Baritone Flying V along with a custom Explorer which he occasionally used on the Locust tour. </fact> <fact> Slash. </fact> <fact> Critics associated his performance with Jimi Hendrixs rendition at Woodstock in 1969, yet Vaughan disliked this comparison I heard they even wrote about it in one of the music magazines and they tried to put the two versions side by side. </fact> <fact> They auditioned a number of singers including onetime Black Flag vocalist Ron Reyes and worked on material that included the main riff of what would become the Guns N Roses song Rocket Queen. </fact> <fact> The following September the band released the longawaited albums Use Your Illusion I and Use Your Illusion II which debuted at No. </fact> <fact> Kiedis observed that his playing evolved during his time away from the group in What Is This with Slovak adopting a more fluid style featuring sultry elements as opposed to his original hard rock techniques. </fact> <fact> It happened Thursday night at Tango. </fact> <fact> ISBN 978-0-470-28906-8cite book  CS1 maint multiple names authors list link
Einarson John with Hillman Chris 2008). </fact> <fact> There was speculation that Pinkly Smooth was going to produce another record but due to the death of the Rev it is highly unlikely that they will produce any more material. </fact> <fact> Burn Halos Dirty Little Girl track and music video. </fact> <fact> You have to rehearse. </fact> <fact> The earliest public example of Townshends involvement with charitable causes was in 1968, when Townshend donated the use of his former Wardour Street apartment to the Meher Baba Association. </fact> <fact> He also thought that settling the case opened a floodgate of claims including the Shape of You lawsuitIn 2018, legal action was brought against Sheeran SonyATV Music Publishing and Atlantic Records by the estate and heirs of the late producer Ed Townsend who cowrote the song Lets Get It On with Marvin Gaye. </fact> <fact> Previously a high school dropout he attended Seattle Universitys Albers School of Business and Economics in the early 2000s and subsequently founded the wealth management firm Meridian Rock. </fact> <plan> He also provided backing vocals for artists such Bowie, Prince Charles William Shatner James Whitlock David Gilmour Keith Richards Peter Gabriels Tony Williams Stephen Carpenter Dixie Dingaman Gary Rossett Steve Howe Phil Rudd Roger Taylor Derek Stavin Brian May
Dyents death has been widely reported over recent months largely due to a story that appeared in The Sun newspaper which claimed he had overdosed following what his family thought were too many champagne fizzy drinks at London nightclub Liberty Hall with fellow musicians including Andrew Lloyd Webber Johnny Rotten Paul McCartney or Janes Addiction side project Duff McKagan leading to injury among other incidents during one particularly drunken night out around 1970. </plan>
<topic> Roland Dyens </topic> <fact> As part of his promotion for the release Harrison performed on Saturday Night Live with Paul SimonIn 1979, Harrison released George Harrison which followed his second marriage and the birth of his son Dhani. </fact> <fact> One evening he disappeared. </fact> <fact> Harry writes that In His Own Write 1964) was published after Some journalist who was hanging around the Beatles came to me and I ended up showing him the stuff. </fact> <fact> The performance was filmed and later released on DVD in September 2004On the following night Double Trouble was booked in the lounge of the Montreux Casino with Jackson Browne in attendance. </fact> <fact> ISBN 979-8838053084. </fact> <fact> In Jealous Guy Lennon addressed his demeaning treatment of women acknowledging that his past behaviour was the result of longheld insecurityIn gratitude for his guitar contributions to Imagine Lennon initially agreed to perform at Harrisons Concert for Bangladesh benefit shows in New York. </fact> <fact> George Harrison said of the album We just played it just wore it out. </fact> <fact> After Nirvana disbanded following the suicide of its frontman Kurt Cobain drummer Dave Grohl went on to become the frontman of Foo Fighters with Smear joining on guitar. </fact> <fact> She also alleged that Gangopadhyay was involved in banning her novel Dwikhandito and her banishment from West Bengal. </fact> <fact> In May 2020, Sheeran donated 170,000 to his former school Thomas Mills High School in Framlingham Suffolk. </fact> <fact> While Albert King had a substantial influence on Vaughan Jimi Hendrix was Vaughans greatest inspiration. </fact> <fact> He was kind of a peculiar fellow. </fact> <fact> Contemporary political pundits described his interpretation as a statement against the Vietnam War. </fact> <fact> Unconditionally followed as the second single from Prism on October 16, 2013, and peaked at number 14 in the United States. </fact> <fact> According to friend Sharon Lawrence liquor set off a bottledup anger a destructive fury he almost never displayed otherwise.In January 1968, the Experience travelled to Sweden to start a oneweek tour of Europe. </fact> <plan> It later emerged that while there police found four small bottles containing cyanide next top left selftitled bluebird jerry canuck lager brand new from Japan only two remained officially identified by their makers but both proved fatal after being opened accidentally exposing them. </plan>
<topic> Roland Dyens </topic> <fact> Her seventh studio album Ray of Light 1998) reflected this change in her perception and image. </fact> <fact> The couple had four children Alice born 1976) Clare born 1979) Sara born 1983) and Matthew born 1986). </fact> <fact> He also thought that settling the case opened a floodgate of claims including the Shape of You lawsuitIn 2018, legal action was brought against Sheeran SonyATV Music Publishing and Atlantic Records by the estate and heirs of the late producer Ed Townsend who cowrote the song Lets Get It On with Marvin Gaye. </fact> <fact> The songs Knock Me Down from Mothers Milk My Lovely Man from Blood Sugar Sex Magik This is the Place from By the Way and Feasting on the Flowers from The Getaway were written about or as tributes to Slovak. </fact> <fact> DuValls first solo album One Alone was released in 2019. </fact> <fact> Ratt and LA. </fact> <fact> Eelpiecom was closed down in 2010 Several of Townshends essays have been posted online including Meher BabaThe Silent Master My Own Silence in 2001, and A Different Bomb an indictment of the child pornography industry the following year. </fact> <fact> ISBN 979-8838053084. </fact> <fact> Machine Heads album The Blackening 2007) climbed to number 54 on the US Billboard after appearing on the charts only two weeks previously it was also nominated for a Grammy Award making it by far the bands most successful album. </fact> <fact> In its review of Dire Straits Brothers in Arms in 1985, Spin commented Mark Knopfler may be the most lyrical of all rock guitarists. </fact> <fact> At about this time Townshend who had been searching the past two years for a basis for a rock opera created a story inspired by the teachings of Baba and other writings and expressing the enlightenment he believed that he had received from them which ultimately became TommyIn interviews Townshend was more open about his beliefs penning an article on Baba for Rolling Stone magazine in 1970 and stating that following Babas teachings he was opposed to the use of all psychedelic drugs making him one of the first rock stars with counterculture credibility to turn against their use. </fact> <fact> In December 2014, he again filed for divorce. </fact> <fact> One of them Let the Mystery Be appeared as the fourth track on 10,000 Maniacs CD single Few and Far Between. </fact> <fact> The earliest public example of Townshends involvement with charitable causes was in 1968, when Townshend donated the use of his former Wardour Street apartment to the Meher Baba Association. </fact> <fact> The following September the band released the longawaited albums Use Your Illusion I and Use Your Illusion II which debuted at No. </fact> <plan> However it now appears more likely than not that this version arose independently although subsequent media reports have suggested its author may have invented part of the legend based on unverified elements discovered separatelyBy late 1971 Eddie who seemed genuinely happy again began experimenting intensely writing psychedelic folk songs inspired partly upon jazz fusion music exemplified earlier under Pat Metheny then further refined into bluesbased psychedelia using synthesizers often built atop old Gibson guitars usually bought off street cornersdealing drugsTowardthe endof 1972 Eddy met vocalist Ronnie Vannucci whose debut album Dreamland featured some early workfor whom Dave Byrd eventually became lead rhythm player replacing Andy Summers leaving Byrnes temporarily without bass until Stewart Douglas took chargeByrne briefly reunitedwith drummer Tom Rhodes before relocating back to England where they recorded three albums named After Dark 1969–1972).

Enlarged Dataset:

In [ ]:
guitarists = ['Jimi Hendrix', 'Eddie Van Halen', 'Eric Clapton', 'B. B. King', 'Jimmy Page', 'Francisco Tárrega', 'Chuck Berry', 'Stevie Ray Vaughan',
              'Slash (musician)', 'Carlos Santana', 'Keith Richards', 'George Harrison', 'Pete Townshend', 'Ana Vidović', 'Andrés Segovia',
              'Billy Idol', 'Leo Brouwer', 'John Mayer', 'Chet Atkins', 'Billie Joe Armstrong', 'Matt Bellamy', 'Ed Sheeran', 'David Gilmour',
              'Gary Green', 'Kurt Cobain', 'Mark Knopfler', 'Elvis Presley', 'Julian Bream', 'Tommy Emmanuel', 'Don Felder', 'Sunil Ganguly',
              'Kirk Hammett', 'Matt Heafy', 'Tony Iommi', 'Robert Johnson', 'Sungha Jung', 'Ed King', 'Chino Moreno', 'Duff McKagan', 'Dave Navarro',
              'Tom Petty', 'Iggy Pop', 'Katy Perry', 'Olivia Rodrigo', 'Johnny Ramone', 'Kapil Srivastava', 'Taylor Swift', 'Hillel Slovak', 'Paul Tobias ',
              'Keith Urban', 'Björn Ulvaeus', 'Vinnie Vincent', 'Jacky Vincent', 'Paul Waaktaar-Savoy', 'Andrew York (guitarist)', 'Chris Martin',
              'Thom Yorke', 'Angus Young', 'Lee Ving', 'John Lennon', 'Fernando Sor', 'Tom Morello', 'Adam Levine', 'Bernie Leadon', 'Henry McCullough',
              'Eric Johnson', 'Tracii Guns', 'Synyster Gates', 'Zach Filkins', 'William DuVall', 'Allen Collins', 'Paul Banks', 'David Byrne', 'Jeff Beck',
              'Mike Campbell', 'David Crosby', 'Bob Dylan', 'Roland Dyens', 'Steve Earle', 'Robb Flynn', 'John Frusciante', 'Rita Hosking', 'Ted Nugent',
              'Adrian Smith', 'Pat Smear', 'Devin Townsend', 'Pete Townshend', 'John Petrucci', 'Steve Jones', 'Steve Hunter', 'Ed O\'Brien', 'Aaron North',
              'Steve Morse', 'Paul McCartney', 'Phil Manzanera', 'Madonna', 'Mark Knopfler', 'Mick Jones (The Clash guitarist)', 'Susanna Hoffs', 'Dan Auerbach']

In [ ]:
with open('/content/drive/MyDrive/LongFromExploration/guitarist_articlesFull.json') as f:
    data = json.load(f)

# Working on Adapting EIPE-Text framework


## Loading Data:
## `MODIFY PATHS AS NECESSARY!!!!`

In [ ]:
# Loading Data:
data = []
with open('/content/drive/MyDrive/LongFromExploration/Guitarist_Dataset_And_Outputs/guitarist_wiki_articles.json') as f:
    data = json.load(f)

### Load Saved content (if saved during testing)

In [ ]:
# Loading Plans and QA pairs IF SAVED:
all_plan_sketches = []
all_qa_pairs = []
file_path = '/content/drive/MyDrive/LongFromExploration/Guitarist_Dataset_And_Outputs/plans_and_qa_pairs.json'

with open(file_path, 'r') as json_file:
    combined_data = json.load(json_file)

all_plan_sketches = combined_data["plan_sketches"]
all_qa_pairs = combined_data["qa_pairs"]

In [ ]:
# Loading all refinement instructions IF SAVED::
all_refinement_instructions = []
file_path = '/content/drive/MyDrive/LongFromExploration/Guitarist_Dataset_And_Outputs/all_refinement_instructions'

with open(file_path, 'r') as json_file:
    all_refinement_instructions = json.load(json_file)

In [ ]:
# Loading refined plans IF SAVED::
refined_plans = []
file_path = '/content/drive/MyDrive/LongFromExploration/Guitarist_Dataset_And_Outputs/refined_plans'

with open(file_path, 'r') as json_file:
    refined_plans = json.load(json_file)

In [ ]:
# Loading generated refined articles IF SAVED::
generated_refined_articles = []
file_path = '/content/drive/MyDrive/LongFromExploration/Guitarist_Dataset_And_Outputs/generated_refined_articles'

with open(file_path, 'r') as json_file:
    generated_refined_articles = json.load(json_file)

In [ ]:
print(refined_plans[0])

# Functions to build Plan Corpus

## Plan sketching:

In [ ]:
def plan_sketching(style_article_text):
    # Instruct GPT-4 to analyze the style of the provided article and create an outline of the topic
    system_prompt = (
        "Distill the salient information and thematic flow from the original article into a tree-like text repre- sentation of a mind map in the following format:"
        "TOPIC - Main Topic - Sub Topic - Sub-Sub Topic - Sub-Sub Topic ..."
    )

    # Provide both the article to analyze and the new topic for the outline
    user_prompt = (
        f"Article to distill salient information from:\n{style_article_text}\n\n"
        f"Generate the outline:"
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        model="gpt-4-turbo",
        max_tokens=1500,  # Increased tokens to account for the analysis and generation tasks
        n=1
    )
    plan_sketch = chat_completion.choices[0].message.content
    return plan_sketch.strip()


## QA Pair Generation:

In [ ]:
# pip install openai==0.28

In [ ]:
# Function to generate QA pairs
def qa_pairs_generation(article_text):
    system_prompt = "Generate several multiple-choice questions and corresponding answers based on the content and length of the article with the following guidelines: 1. Not too detailed, 2. Focus on the logic of the article, 3. Deep understanding of the article after answering these questions, 4. Each question must have 4 options: A, B, C, D., 5. For each question, there might be more than one correct answer, identify all correct an- swers separated by ;, 6. Questions should reflect the structure of the article., 7. Questions should include three types: what, why, how., 8. Provide related main ideas in the article for each question., 9. Avoid options like All of the above or None of the above; use A;B;C format. These questions are generated based on the article’s content and the author’s opinion, not my opinion. "
    user_prompt = f"Based on the content of the following article, generate several multiple-choice questions and corresponding answers: {article_text}"

    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        model="gpt-4-turbo",
        max_tokens=1024,
        n=1
    )
    qa_pairs = chat_completion.choices[0].message.content
    return qa_pairs.strip()


### Test function code:

In [ ]:
len(data[3])
# data[1][21000:]

23130

In [ ]:
# CAREFUL WHEN RUNNING - Costs about $.20
article_text = data[3]
plan_sketch = plan_sketching(article_text)
print("Generated Plan Sketch:\n", plan_sketch)

In [ ]:
# CAREFUL WHEN RUNNING - Costs about $0.20
qa_pairs = qa_pairs_generation(article_text)
print("Generated QA Pairs:\n", qa_pairs)

In [ ]:
all_plan_sketches = []
all_qa_pairs = []

In [ ]:
file_path = '/content/drive/MyDrive/LongFromExploration/plans_and_qa_pairs.json'

with open(file_path, 'r') as json_file:
    combined_data = json.load(json_file)

all_plan_sketches = combined_data["plan_sketches"]
all_qa_pairs = combined_data["qa_pairs"]

In [ ]:
# all_plan_sketches.append(plan_sketch)
# all_qa_pairs.append(qa_pairs)
print(len(all_plan_sketches))
print(len(all_qa_pairs))

4
4


In [ ]:
print(all_plan_sketches[0])

In [ ]:
# combined_data = {
#     "plan_sketches": all_plan_sketches,
#     "qa_pairs": all_qa_pairs
# }
# file_path = '/content/drive/MyDrive/LongFromExploration/plans_and_qa_pairs.json'
# with open(file_path, 'w') as json_file:
#     json.dump(combined_data, json_file)

## QA Based Evaluation:

In [ ]:
def evaluate_plan_on_qapairs(plan, qa_pairs):
    """
    Parameters:
    - plan (str): The current plan for the article.
    - qa_pairs (str): Generated QA pairs used to evaluate the plan.

    Returns:
    - bool: Indicates whether modifications are necessary.
    - str: Refinement instructions from GPT-4.
    """
    system_prompt = (
        "Evaluate the plan based on the following QA pairs. Provide detailed refinement instructions for any incorrect information identified by the QA pairs. "
        "The modification instruction can be one of the following: (1) add, which inserts a missing node into the plan; (2) modify, which alters the content of a node; "
        "(3) adjust, which relocates a node to another level of the tree. Provide precise improvements to specific parts of the plan."
    )

    user_prompt = f"Based off this plan and QA pairs, provide what modifications are necessary. Plan:\n{plan}\n\nQA Pairs:\n{qa_pairs}"

    chat_completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1500,
        n=1,
    )

    evaluation_result = chat_completion.choices[0].message.content.strip()

    # Determine if modifications are necessary based on the response
    modifications_needed = "no modifications necessary" not in evaluation_result.lower()
    return modifications_needed, evaluation_result



### Test code for function:

In [ ]:
all_refinement_instructions = []

In [ ]:
file_path = '/content/drive/MyDrive/LongFromExploration/all_refinement_instructions'
with open(file_path, 'r') as json_file:
    all_refinement_instructions = json.load(json_file)

In [ ]:
# Example usage with dummy plan and QA pairs
plan = all_plan_sketches[3]
qa_pairs = all_qa_pairs[3]

evaluation_result = evaluate_plan_on_qapairs(plan, qa_pairs)
print("Evaluation Result:\n", evaluation_result)

In [ ]:
# all_refinement_instructions.append(evaluation_result)
print(all_refinement_instructions[0])

In [ ]:
print(len(all_refinement_instructions))

0


In [ ]:
file_path = '/content/drive/MyDrive/LongFromExploration/all_refinement_instructions'
with open(file_path, 'w') as json_file:
    json.dump(all_refinement_instructions, json_file)

## Plan Refinement:


In [ ]:
def plan_refinement(plan, refinement_instructions):
    """
    Function to instruct GPT-4 to apply all refinement instructions to the plan.
    """
    user_prompt = (
        f"Here is the current plan:\n{plan}\n\n"
        f"Apply the following refinement instructions to improve the plan:\n{refinement_instructions}\n\n"
        f"Provide the updated plan after applying all instructions in the same format as it was orinally presented."
    )

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": user_prompt}],
        model="gpt-4-turbo",
        max_tokens=1024,
        n=1,
    )

    # Assuming the model's response contains the updated plan
    updated_plan = chat_completion.choices[0].message.content
    return updated_plan.strip()


### Test code for function:

In [ ]:
curr_refined_plan = plan_refinement(all_plan_sketches[3], all_refinement_instructions[3])
print('Refined Plan:\n', curr_refined_plan)

In [ ]:
refined_plans = []

In [ ]:
refined_plans.append(curr_refined_plan)

In [ ]:
print(len(refined_plans))

4


In [ ]:
with open('/content/drive/MyDrive/LongFromExploration/refined_plans', 'w') as json_file:
    json.dump(refined_plans, json_file)

# MAIN LOOP

In [ ]:
def refine_plan_loop(initial_article_text):
    """
    Refines the plan for the article text by iteratively generating QA pairs and applying modifications.
    Parameters:
        initial_article_text (str): The text of the initial article.
    Returns:
        str: The final refined plan.
    """
    plan = plan_sketching(initial_article_text)
    max_iterations = 4
    min_iterations = 2
    iteration = 0
    modifications_needed = True

    while iteration < max_iterations and (modifications_needed or iteration < min_iterations):
        qa_pairs = qa_pairs_generation(initial_article_text)

        # Evaluate if modifications are needed based on the QA pairs generated
        modifications_needed, refinement_instructions = evaluate_plan_on_qapairs(plan, qa_pairs)

        if not modifications_needed and iteration >= min_iterations:
            print(f"No further modifications needed after {iteration + 1} iterations.")
            break

        plan = plan_refinement(plan, refinement_instructions)
        iteration += 1
        print(f"Iteration {iteration}: Plan refined.")

    print("Refinement process completed.")
    return plan


# Code to generate output for novel test based off Plan Corpus:

In [ ]:
import openai

def generate_plan(topic, example_plans, model="gpt-4-turbo"):
    """
    Generate a new plan on a given topic using provided example plans from a corpus.

    Parameters:
        topic (str): The topic for the new plan.
        example_plans (str): String containing several example plans from the corpus.
        model (str): Model identifier for the OpenAI API.

    Returns:
        str: A newly generated plan for the topic.
    """
    system_prompt = "Generate a detailed plan on the given topic using the style and structure of the example plans provided below."
    user_prompt = f"Topic: {topic}\n\nExamples for style reference:\n{example_plans}\n\nNew plan:"

    chat_completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=500
    )
    return chat_completion.choices[0].message.content.strip()


In [ ]:
def create_passage(plan, model="gpt-4-turbo"):
    """
    Generate a detailed article based on the provided plan.

    Parameters:
        plan (str): The refined plan to base the article on.
        model (str): Model identifier for the OpenAI API.

    Returns:
        str: The generated article as a string.
    """
    system_prompt = "Generate a detailed article based on the following plan. Ensure the article reflects the style and structure embedded within the plan."
    user_prompt = f"Refined Plan:\n{plan}\n\nWrite the article:"

    chat_completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1500
    )
    return chat_completion.choices[0].message.content.strip()


## Guitarist Bios Wiki testing:

In [ ]:
guitaristwiki_refined_plan_corpus = []

In [ ]:
# LOAD CORRESPONDING PLAN CORPUS HERE IF SAVED:
# with open("/content/drive/MyDrive/LongFromExploration/guitaristwiki_refined_plan_corpus.json", 'r') as f:
#     guitaristwiki_refined_plan_corpus = json.load(f)

In [ ]:
# LOOP TO CREATE PLAN CORPUS
for i in range(0,7):
  guitaristwiki_refined_plan_corpus.append(refine_plan_loop(data[i]))

In [ ]:
print(guitaristwiki_refined_plan_corpus[4])

## Testing Novel Outputs for Guitarist Wiki Dataset:

In [ ]:
example_plans = "\n\n".join(f"Example Plan {i+1}:\n{plan}" for i, plan in enumerate(guitaristwiki_refined_plan_corpus))
new_plan = generate_plan("Ed Sheeran", example_plans)

In [ ]:
print(new_plan)

In [ ]:
novel_passage = create_passage(new_plan)

In [ ]:
print(novel_passage)

## Test 1: Carlos Santana

### Carlos Santana: The Fusion Maestro of Rock and Latin American Music

**Introduction**

Carlos Santana, with his inimitable fusion of rock, Latin music, and jazz contours, has remained a formidable name in the world of music. His distinctive sound, marked by heartfelt guitar solos and rhythmically complex components, not only defines a unique musical identity but also mirrors the cultural diversity that he sprang from. Santana's contribution to both rock and Latin American music genres has been profound, his influence long-standing.

**Biography**

**Early Life**

Born on July 20, 1947, in Autlán de Navarro, Jalisco, Mexico, Carlos was introduced to music at a tender age by his father, a skilled mariachi musician. This early introduction to traditional mariachi sounds deeply embedded the essence of Latin music in Santana’s future creations. His family's move to Tijuana and then to San Francisco in the early 1960s exposed him to a melting pot of musical styles, which he would later synthesize in his own musical expression.

**Career Development**

1966 marked the formation of the Santana Blues Band, which was later simplified to "Santana". His breakthrough came with a spellbinding performance at the Woodstock Festival in 1969, which opened doors to a global audience. The 1970 release of "Abraxas", with hits like "Black Magic Woman" and "Oye Como Va," catapulted him to rock stardom. This album's success underscored his ability to blend rock with Latin jazz and funk seamlessly.

**Musical Style and Influences**

Carlos Santana's style is a rich tapestry woven with threads of Latin American rhythms, rock stability, and jazz improvisation. His guitar, often an extension of his vivid expressions, uses sustained notes that pierce through the texture of his band's sound, evoking deep emotional responses. His music, while rooted in blues and American rock, bows significantly to traditional Latin forms, bearing testimony to his heritage and his innovative genius alike.

**Key Albums and Songs**

**Early Works**

His debut album "Santana" (1969), featured "Soul Sacrifice," a track that became emblematic of his style and ethos. With "Abraxas" (1970), Santana established his international appeal, melding diverse musical influences into a coherent, electrifying style.

**Later Works**

Decades later, the album "Supernatural" (1999) marked a major comeback, resonating with a wider audience through collaborations with contemporary stars like Rob Thomas on the track "Smooth" and "Maria Maria". This blend of the old and the new brought fresh vitality to his career, drawing in a younger fanbase while satisfying long-time followers.

**Awards and Recognitions**

Carlos Santana's shelves are adorned with multiple Grammy Awards; notably, he tied the record for most Grammys won in a single night for "Supernatural." His revolutionary guitar work has also earned him a place in Rolling Stone’s list of the Greatest Guitarists of All Time, cementing his legacy in the annals of rock history.

**Cultural and Social Impact**

As one of the first Latin rock musicians to find widespread success in the United States, Santana broke cultural barriers, paving the way for future generations of Latin American musicians. His achievements challenged the norms of the rock genre and broadened the American musical landscape to include vibrant and varied cultural influences.

**Personal Life and Philosophy**

Deeply spiritual, Santana’s philosophies and personal beliefs deeply influence his music and way of life. His journey through evolving musical landscapes is also a quest for inner peace and enlightenment, facets frequently reflected in his lyrics and the ethos of his projects.

**Conclusion**

Carlos Santana remains not just a musical icon but a bridge between varied cultures and epochs. Through his strings, he tells a story of integration, innovation, and inspiration, making him a pivotal figure in the chronicles of modern music. Saluting his heritage while embracing global influences, Santana continues to orchestrate a symphony of musical unity and artistic prowess.

## Test 2: Ed Sheeran

### **Ed Sheeran: A Modern Musical Phenomenon**

#### **Introduction**
In the landscape of contemporary music, few artists have made as indelible a mark as Ed Sheeran. Known for his melodic pop songs that are heavily influenced by folk and acoustic music, Sheeran stands out as a pivotal figure in the music industry. Not only is he celebrated for his emotive vocals and heartfelt lyrics, but his prowess as a record producer and songwriter further cements his status among musical elites.

#### **Early Life and Musical Beginnings**
**Birth and Family:**
Ed Sheeran was born on February 17, 1991, in Halifax, West Yorkshire, England, and was raised in the quaint town of Framlingham, Suffolk. His upbringing in a creative, art-loving family set the foundation for his musical journey.

**Early Interest in Music:**
From the tender age of four, Sheeran began singing in his local church choir, hinting at the early blossoming of his musical talent. He picked up the guitar during his school years and started to pen his own songs, a habit that would soon pave the way for his future career.

**Education and Early Career:**
Ed attended Thomas Mills High School, where he began to seriously nurture his musical talent. After completing his schooling, Sheeran made a bold move to London at 18. Here, he played in small venues and slowly built a follower base through online performances, an endeavor that set the stage for his later breakthroughs.

#### **Career Highlights**
**Breakthrough and Rise to Fame:**
Sheeran's career trajectory changed with the release of the independent EP, "No. 5 Collaborations Project." This collection not only showcased his versatility and innate knack for blending genres but also caught the attention of music legends like Elton John and Jamie Foxx. His major label debut album, "+," arrived in 2011 featuring the poignant single "The A Team," which garnered critical acclaim and commercial success.

**Continued Success:**
Sheeran's ascent continued with his 2014 album "x" (Multiply), which included chart-dominating hits such as "Thinking Out Loud." His third album, "÷" (Divide) released in 2017, featured the viral singles "Shape of You" and "Perfect," songs that played no small part in Sheeran capturing hearts globally.

**Global Tours and Performances:**
Notable among his numerous live performances is headlining the prestigious Glastonbury Festival in 2017. His "Divide Tour," spanning several continents, eventually became the highest-grossing tour of all time, a testament to his widespread appeal and robust fan base.

#### **Musical Style and Influences**
**Artistic Style:**
Sheeran is particularly noted for his skillful guitar work, seamlessly weaving elements of pop, folk, and even hip-hop. During live performances, he often uses a loop pedal to enhance his one-man shows, creating intricate, multi-layered sounds that captivate audiences.

**Lyrical Themes:**
His lyrics frequently draw from personal experiences, addressing themes of love, loss, and reflection, alongside poignant societal commentary. This relatability is partly what draws listeners to his music.

**Influences:**
Sheeran's musical influences are diverse, ranging from the soulful melodies of Van Morrison to the lyrical depth of Damien Rice and the dynamic energy of Eminem.

#### **Personal Life and Philanthropy**
**Family and Relationships:**
Ed Sheeran's personal life, particularly his marriage to Cherry Seaborn, a childhood friend, harmonizes with his public persona. The couple, now parents to two daughters, shares a life away from the limelight in Suffolk.

**Philanthropic Efforts:**
Complementing his musical career are Sheeran's charitable activities. His efforts are broad-ranging, focusing on music education, children's hospices, and aiding the underprivileged, showcasing his dedication not just to music but humanitarian causes as well.

As he continues to evolve both personally and professionally, Ed Sheeran's journey is a resonant reminder of the power of passion and perseverance in the pursuit of musical and personal fulfillment. His story is far from over, and the music world eagerly anticipates his next chapter.

## Medline Testing:

In [ ]:
import pandas as pd

In [ ]:
# LOAD MEDLINE PREPROCESSED TEXT DATA HERE:
medline_data = pd.read_csv("/content/drive/MyDrive/LongFromExploration/medline_data.csv")

In [ ]:
print(medline_data)

In [ ]:
medline_refined_plan_corpus = []

In [ ]:
with open("/content/drive/MyDrive/LongFromExploration/medline_refined_plan_corpus.json", 'r') as f:
    medline_refined_plan_corpus = json.load(f)

In [ ]:
# MAIN TRAINING LOOP:
## Modify max length from 4 to desired size, but be careful since each iteration will call
## GPT API atleast 5 times. Can also modify parameter to stop refine loop in function above
for i in range(0, 4):
    # Get the text from the 'text' column
    article_text = medline_data.iloc[i]['text']
    refined_plan = refine_plan_loop(article_text)
    medline_refined_plan_corpus.append(refined_plan)

In [ ]:
print(len(medline_refined_plan_corpus))

In [ ]:
print(medline_refined_plan_corpus[3])

In [ ]:
example_plans = "\n\n".join(f"Example Plan {i+1}:\n{plan}" for i, plan in enumerate(medline_refined_plan_corpus))
new_plan = generate_plan("Midostaurin", example_plans)

In [ ]:
print(new_plan)

In [ ]:
novel_passage = create_passage(new_plan)

In [ ]:
print(novel_passage)

## Test 1: Chlorpropamide

### Chlorpropamide: An Essential Tool in Managing Type 2 Diabetes

#### Overview

**Definition**: Chlorpropamide is a medication primarily used to manage Type 2 diabetes mellitus. It belongs to a class of drugs known as sulfonylureas, which are pivotal in the treatment of this chronic condition.

**Importance**: The drug plays a crucial role in controlling blood sugar levels. It does so by stimulating the release of insulin from the pancreas—a critical process for those whose bodies do not produce enough insulin naturally. Moreover, it enhances the body’s sensitivity to insulin, thus aiding in the effective utilization of available insulin.

#### Prescribed For

**Main use**: Chlorpropamide is specifically designated for adult patients with Type 2 diabetes, especially those who have not achieved desired blood sugar control through diet, exercise, and other medications. It is particularly useful in scenarios demanding a potent or long-acting sulfonylurea.

**Considerations**: Due to its efficacy in lowering blood sugar levels, healthcare providers often prescribe it when other treatments fall short.

#### Drug Classification

**Type of medication**: As a sulfonylurea hypoglycemic agent, Chlorpropamide operates by reducing blood glucose levels.

**Category**: It is categorized under medications that regulate blood sugar, vital for the prevention of diabetes-related complications.

#### Mechanism of Action

**How it works**: The primary function of Chlorpropamide is to lower glucose levels in the blood. It prompts pancreatic beta cells to produce more insulin and enhances the insulin sensitivity of peripheral tissues. This dual action not only manages but can also significantly improve blood sugar levels.

**Pharmacodynamics**: In addition to its insulin-stimulating abilities, Chlorpropamide may also curb hepatic glucose production. This additional function helps to further reduce blood sugar levels, providing a comprehensive treatment approach.

#### Administration Guidelines

**Dosage and administration**: Chlorpropamide is taken orally, typically once daily, and is best started at low dosages to mitigate the risk of hypoglycemia—a common side effect where blood sugar levels fall too low.

**Diet and lifestyle adjustments**: It is highly recommended that the treatment plan for diabetes include not only medications like Chlorpropamide but also proper dietary habits and regular physical activity. Such holistic approaches enhance the medication's effectiveness and overall health.

#### Risks and Side Effects

**Common side effects**: While Chlorpropamide is effective, it can cause several side effects such as weight gain, nausea, dizziness, and allergic skin reactions. Hypoglycemia, although a common side effect, requires immediate attention and appropriate management.

**Severe reactions**: The drug can induce severe hypoglycemia, particularly in older adults or those with impaired kidney function. Additionally, it may react with alcohol, leading to disulfiram-like reactions—nausea, vomiting, and palpitations.

**Long-term use considerations**: Some studies have suggested an increased risk of cardiovascular deaths associated with long-term use of Chlorpropamide compared to dietary management or other antidiabetic therapies.

#### Special Precautions

**Age and dosage adjustments**: Special consideration is recommended for elderly patients—typically starting at lower doses—to decrease the likelihood of hypoglycemia.

**Renal and liver function**: Patients with compromised kidney or liver function must be closely monitored due to the prolonged half-life of the drug in such conditions, potentially necessitating dosage adjustments.

#### Interactions

**With other drugs**: The likelihood of hypoglycemia is notably heightened when Chlorpropamide is used in conjunction with other antidiabetic medications, as well as with substances like alcohol, beta-blockers, or NSAIDs.

**Dietary interactions**: The interaction of Chlorpropamide with certain foods can affect its efficacy and safety, highlighting the importance of discussing dietary habits with a healthcare provider.

In conclusion, Chlorpropamide remains a valuable medication for managing Type 2 diabetes, particularly in cases where other methods have proven insufficient. However, it requires careful consideration of its side effects, interactions, and long-term use implications to ensure it is used safely and effectively.

## Test 2: Quinine

### Quinine: The Historical and Contemporary Significance of an Anti-Malarial Warrior

Quinine, a natural compound extracted from the bark of the cinchona tree, remains pivotal in medical history for its role in treating malaria, among other conditions. This bitter alkaloid has shaped medical practices, especially before the advent of synthetic anti-malarial drugs, and continues to have significant medical and historical relevance.

#### Understanding Quinine: Definition and Importance
Quinine is primarily celebrated for its anti-malarial properties. It disrupts the life cycle of malaria parasites in human blood, leading to the elimination of the disease in infected individuals. The singular ability of quinine to combat malaria, a disease that has plagued humanity for centuries, signifies its crucial role in medicine and its substantial impact on public health.

#### Historical Context: From Indigenous Remedy to Global Medicine
Quinine's journey from a traditional South American remedy to a staple in European medicine is a fascinating tale of intercontinental botanical exchange. Originally utilized by indigenous tribes in the Andes, quinine was introduced to European settlers in the 17th century. Its adoption transformed it into a tool of empire, facilitating the expansion of European powers into the malaria-ridden terrains of Africa and Asia by protecting colonists and military personnel from the ravages of the disease.

#### Medical Uses Beyond Malaria
While the primary application of quinine is in treating malaria, its use is versatile:
- **Malaria Treatment**: Administered in various forms such as tablets and intravenous injections, quinine remains a frontline defense against malaria, particularly in areas where resistance to newer drugs is prevalent.
- **Treatment of Leg Cramps and Other Conditions**: Although not FDA-approved for such uses due to safety concerns, quinine has been explored for treating nocturnal leg cramps and other inflammatory conditions due to its muscle relaxant properties.

#### Quinine’s Mechanism and Drug Classification
As an antimalarial agent, quinine interferes with the reproduction and protein synthesis of malaria parasites. Despite its efficacious profile, it is classified strictly under FDA guidelines which approve its usage mainly for malaria, with strict regulations due to potential adverse effects when used for other conditions.

#### Side Effects and Risks
Quinine's application, while life-saving, is not without risks. Common side effects include nausea and vision disturbances, but more severe reactions such as arrhythmias and severe hypoglycemia can occur, particularly with high doses or prolonged use. Patients with conditions like tinnitus or optic neuritis are advised against its use due to higher susceptibility to its adverse effects.

#### Contemporary Relevance and Challenges
Today, quinine’s utility is marred somewhat by the emergence of drug-resistant strains of malaria. This phenomenon challenges the effectiveness of quinine, narrowing its once broad efficacy. Modern research is heavily invested in chemically modifying quinine to counteract resistance and extend its relevancy as an anti-malarial agent.

#### Conclusion
Quinine's historical and medical significance cannot be understated. From reducing the impact of malaria to enabling colonial ventures into hostile territories, its impact on human history is profound. Despite challenges such as drug resistance and significant side effects, quinine continues to be an essential tool in the global fight against malaria. As research progresses, there is hope that quinine can be adapted to overcome current challenges, securing its place in the pharmacopeia of tomorrow.

# Test 3: Hydroxychloroquine

### Understanding Hydroxychloroquine: Uses, Mechanisms, and Guidelines

**Overview**

Hydroxychloroquine is primarily recognized for its application in both the prevention and treatment of malaria and its substantial benefits in managing autoimmune diseases, such as lupus and rheumatoid arthritis. Its significance is rooted in its dual functionality, which makes it a critical pharmaceutical agent in many parts of the world where malaria is endemic and autoimmunity rates are significant.

**Medicinal Uses**

1. **Malaria**: Hydroxychloroquine stands out in the fight against malaria, especially against chloroquine-sensitive strains. By preventing and treating this mosquito-borne disease, it contributes substantially to public health efforts in malaria-prone areas.
   
2. **Autoimmune Diseases**:
    - **Rheumatoid Arthritis**: The drug is beneficial in mitigating chronic symptoms like joint stiffness, swelling, and pain associated with rheumatoid arthritis, offering sufferers a better quality of life.
    - **Systemic Lupus Erythematosus (SLE)**: For those battling SLE, hydroxychloroquine is a cornerstone in managing both skin issues and arthritic symptoms, substantially reducing disease flares and enhancing life management under the shadow of this condition.

3. **Controversial Use in COVID-19**: Initially, hydroxychloroquine was touted as a potential game-changer in the fight against COVID-19, leading to widespread use. However, subsequent studies have shown mixed efficacy results, prompting health authorities to advise against its routine use for COVID-19 outside of clinical trials.

**Drug Classification**

Hydroxychloroquine is categorized as a disease-modifying anti-rheumatic drug (DMARD) and an antimalarial agent. Its versatility is due to its ability to modify underlying disease processes.

- **Mechanism of Action**:
  - **Immunomodulatory Effects**: In autoimmune diseases, hydroxychloroquine modulates the immune system, adjusting its overactivity, which is crucial in reducing symptomatic flares.
  - **Antimalarial Effects**: The drug increases the pH within the intracellular vacuoles of the malaria parasite. This change hampers the parasite’s capability to process and utilize hemoglobin, effectively stifling its lifecycle.

**Administration Guidelines**

The administration of hydroxychloroquine requires cautious handling:
- **Dosage**: The dosage of hydroxychloroquine is highly variable, dependent on medical guidance, the specific condition in question, and patient body weight.
- **Form**: It is most commonly prescribed in oral tablet form.
- **Precautionary Measures**: Given the potential for severe side effects, regular monitoring is paramount, particularly regarding eyesight to detect any early signs of retinal toxicity, and cardiac monitoring to prevent arrhythmias.

**Side Effects and Risks**

While hydroxychloroquine is generally well-tolerated, it does come with potential side effects:
- **Common Side Effects**: These include nausea, abdominal cramps, and dizziness – symptoms that are typically manageable.
- **Severe Risks**: More serious risks involve retinal damage, particularly consequential in long-term usage, cardiac irregularities such as arrhythmias, pronounced drops in blood sugar levels, and possible severe exacerbation of psoriasis.
- **Monitoring**: Ophthalmologic evaluation is recommended periodically to preempt visual impairment. Furthermore, cardiac assessments are advocated to foreclose substantial heart-related complications.

In conclusion, hydroxychloroquine, while versatile and effective for certain conditions, mandates careful usage under professional guidance to balance its benefits against potential severe adverse effects, ensuring it serves its intended beneficial roles without undue risks.

# Test 4: Pentazocine

**Understanding Pentazocine: A Comprehensive Insight into a Unique Opioid**

Pentazocine is a synthesized opioid analgesic that plays an essential role in modern medicine, primarily known for managing moderate to severe pain. It can be used independently or as part of anesthesia protocols. Its distinctive properties offer a balance between effective pain relief and a minimized risk of dependence, distinguishing it from stronger opioids.

**Detailed Drug Classification**

Pentazocine’s unique interaction with opioid receptors classifies it as an opioid agonist-antagonist. It functions as an agonist at kappa receptors while simultaneously acting as an antagonist at mu receptors. Due to its potential for abuse, though notably lower than that of higher schedule opioids, it is classified as a Schedule IV controlled substance in the United States, highlighting the need for careful prescription and handling.

**Indications for Use**

In clinical settings, pentazocine is primarily indicated for patients suffering from moderate to severe pain when non-opioid analgesics fail to provide relief. Additionally, it serves as an important component in anesthesia, used in combination with other agents. Its employment as an alternative to more potent opioids makes it particularly valuable for individuals prone to, or at risk of developing, opioid dependency.

**Mechanism of Action**

The mechanism of action of pentazocine involves dual activity at different opioid receptors. It primarily provides pain relief through activation of kappa opioid receptors. This mechanism does not lead to the euphoria commonly associated with mu receptor agonism, which is typical of stronger opioids. Moreover, its antagonistic properties at the mu receptors help curb the effects of other opioids, thereby potentially mitigating the likelihood of developing dependence.

**Side Effects and Risks**

The administration of pentazocine, like all medications, comes with potential side effects. Commonly observed adverse effects include nausea, dizziness, sweating, and drowsiness. However, it also poses risks of more severe reactions, such as respiratory depression, hallucinations, and dependency—albeit at a lower risk comparatively. It is contraindicated in patients with a history of opioid addiction or those currently on full opioid agonists, as it can precipitate withdrawal symptoms.

**Dosage Forms**

Pentazocine is available in various forms, including tablets and injections. This versatility allows healthcare providers to tailor the administration method to best fit the patient’s medical condition and management plan, ensuring both efficacy and convenience.

**Comparative Analysis**

When compared to full agonist opioids, pentazocine offers a distinctly lower risk of abuse and dependency. However, it is potentially less efficacious in managing extreme pain conditions. Its utility in chronic pain management is limited due to the risk of adverse psychotropic effects, making it less favorable for long-term use compared to other analgesics.

**Regulatory and Legal Status**

The regulatory framework surrounding pentazocine is stringent, governed by both national and international guidelines that regulate opioid prescriptions. These control measures are crucial for preventing misuse and managing the public health aspects associated with opioid medications.

In conclusion, pentazocine represents a critical component of pain management with its unique pharmacological profile offering a significant therapeutic alternative in the opioid category. Its potential for lower dependency risks, combined with effective pain control, positions it as a viable option for many patients, particularly those at risk of opioid abuse. Nevertheless, its use must be carefully monitored and tailored to individual patient profiles to maximize benefits and minimize risks.

# Test 5: Thiothixene

### Understanding Thiothixene: An Antipsychotic Medication for Schizophrenia

**Overview**

Thiothixene is a medication primarily used to manage symptoms associated with schizophrenia. Defined as an antipsychotic drug, thiothixene plays a vital role in the treatment of psychotic disorders, helping control severe symptoms such as delusions, hallucinations, and other disruptive behaviors. This medicinal approach is a cornerstone in psychiatric treatments that aim to improve quality of life for those affected by profound mental health disturbances.

**Indications**

Primarily, thiothixene is prescribed for the treatment of schizophrenia, a chronic and severe mental disorder that affects how a person thinks, feels, and behaves. Often characterized by episodes where patients may lose touch with reality, schizophrenia is complex, demanding a robust therapeutic strategy, for which thiothixene is crucial. Besides its primary use, thiothixene is also utilized off-label to manage severe anxiety and behavioral disorders, though these uses are based on the discretion and clinical judgement of healthcare providers.

**Drug Classification**

Classified within the umbrella of typical antipsychotics, thiothixene falls into the subcategory of thioxanthenes. Thioxanthenes are distinctive due to their influence on neurotransmitter pathways, particularly affecting the brain's chemical messengers that significantly impact mood, perception, and alertness.

**Mechanism of Action**

The efficacy of thiothixene comes from its action on neurotransmitter receptors, specifically targeting dopamine D2 receptors. Dopamine, a crucial neurotransmitter, is oftentimes abnormally elevated in disorders like schizophrenia. By blocking these receptors, thiothixene helps normalize the levels and transmission of dopamine, which in turn, mitigates the psychotic symptoms commonly seen in schizophrenia patients.

**Dosage Forms**

Thiothixene is most commonly available as oral capsules that come in various strengths. Dosage generally begins at a lower spectrum and is carefully adjusted based on the therapeutic response of the patient alongside considerations for potential side effects.

**Side Effects**

While thiothixene is effective in symptom control, it is not devoid of side effects. Common adverse effects include drowsiness, dizziness, dry mouth, nausea, and constipation, typically manageable and likely to decrease over time. However, it may also cause severe side effects like Extrapyramidal symptoms (EPS), tardive dyskinesia, and perhaps most dangerously, neuroleptic malignant syndrome (NMS). These conditions require immediate medical attention due to their potentially life-threatening nature.

**Precautionary Measures**

Patients with pre-existing conditions such as liver or kidney disease, cardiovascular issues, or those with a history of seizures should exercise caution when taking thiothixene. Regular monitoring by healthcare providers is essential to track side effects, particularly those related to movement disorders and metabolic changes.

**Interactions**

Thiothixene's interaction with other drugs and substances can affect its efficacy and safety. Crucial interactions include those with CYP450 inducers and inhibitors, which might alter thiothixene's metabolism, and CNS depressants, which could amplify sedative effects leading to increased risk of sedation or respiratory depression. Patients are advised to avoid alcohol while on thiothixene due to its potential to enhance adverse effects like drowsiness and dizziness.

**Patient Information**

For optimal results, thiothixene should be taken with food to mitigate gastrointestinal side effects. Adherence to the prescribed regimen is crucial, as is maintaining regular consultations with a physician for monitoring and adjusting treatment as necessary.

Thiothixene, by aiding in the control of schizophrenia symptoms, plays an indispensable role in mental health care, enabling patients to lead more stable and fulfilling lives. However, informed use, including awareness of potential interactions and vigilance for side effects, is key to maximizing its benefits and minimizing risks.

# Test 6: Midostaurin

### Understanding Midostaurin: A Promising Anti-Cancer Medication

**Overview**  
Midostaurin is a specialized anti-cancer medication used primarily for targeting specific types of blood cancers, including acute myeloid leukemia (AML) and systemic mastocytosis. Its mechanism focuses on a targeted therapy approach, which is pivotal in cancer treatment today as it allows for the selective targeting of cancerous cells while minimizing damage to healthy cells. This specificity not only enhances the effectiveness of the treatment but also reduces the severity of associated side effects.

**Importance**  
The significance of midostaurin lies in its ability to interfere with the pathways that cancer cells use to grow and proliferate. For patients with certain genetic mutations, such as the FLT3 mutation in AML, midostaurin represents a critical advancement in their treatment options, offering a new hope for better outcomes.

**Indications**  
Midostaurin serves two primary roles in the treatment of blood cancers:
- **Acute Myeloid Leukemia (AML)**: It is specifically approved for use in AML patients who have an FLT3 mutation. This mutation is associated with particularly aggressive forms of the cancer. Before initiating treatment with midostaurin, patients must undergo a companion diagnostic test to confirm the presence of FLT3 mutations.
- **Aggressive Systemic Mastocytosis/Mast Cell Leukemia**: For these rare disorders, midostaurin is employed when patients do not adequately respond to conventional treatment methods.

**Drug Classification**  
Classified as a kinase inhibitor, midostaurin effectively targets FLT3 kinase among other tyrosine kinases. FLT3 kinase is a protein that is critical for the survival and multiplication of malignant blood cells in certain types of blood cancers.

**Mechanism of Action**  
The core of midostaurin’s action is its inhibitory effect on various receptor tyrosine kinases, including the FLT3. By inhibiting these kinases, midostaurin disrupts important cellular signaling pathways that are essential for the growth and survival of cancer cells, thereby inhibiting tumor growth and proliferation.

**Treatment Procedure**  
Midostaurin is administered in combination with standard chemotherapy regimens. Treatment typically involves:
- **Initial Induction Phase**: Where the goal is to achieve remission from the active cancer.
- **Consolidation Phase**: This phase ensures the thorough destruction of any remaining cancer cells, followed by a maintenance therapy designed to prevent cancer relapse.

**Clinical Benefits**  
Studies and clinical trials have consistently shown that midostaurin, when combined with chemotherapy, significantly improves survival rates in adults with FLT3-mutated AML. Furthermore, in systemic mastocytosis, midostaurin helps manage severe symptoms and enhances the patients’ quality of life by reducing the systemic burden of the disease.

**Risks and Side Effects**  
Like all cancer treatments, midostaurin has its share of potential side effects. Commonly observed adverse effects include nausea, vomiting, diarrhea, and febrile neutropenia. Patients are closely monitored for these effects, and supportive care is provided to manage them effectively.

**Conclusion**  
Midostaurin represents a beacon of hope for patients with specific types of blood cancers. By focusing on mutations specific to individual patients, it not only offers a more effective treatment but also a personalized approach to cancer therapy. As research continues and treatment protocols refine, midostaurin’s role in cancer care is set to become even more significant, helping countless patients achieve better outcomes and improved quality of life.

# Saving Code:

In [ ]:
file_path = '/content/drive/MyDrive/LongFromExploration/medline_refined_plan_corpus.json'
# Write the list to a JSON file
with open(file_path, 'w') as f:
    json.dump(medline_refined_plan_corpus, f)

In [ ]:
file_path = '/content/drive/MyDrive/LongFromExploration/guitaristwiki_refined_plan_corpus.json'
# Write the list to a JSON file
with open(file_path, 'w') as f:
    json.dump(guitaristwiki_refined_plan_corpus, f)